In [1]:
from tqdm import tqdm
import re
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

/storage/zjwu/anaconda3/envs/cuda11/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_train = pd.read_csv("./stock_dataset/Combined_train.csv")
df_valid = pd.read_csv("./stock_dataset/Combined_valid.csv")
df_test = pd.read_csv("./stock_dataset/Combined_test.csv")

In [3]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
DistilBertModel = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [4]:
def combine_text(df, tokenizer):
    headline=[]
    for row in range(0,len(df.index)):
        headline.append(" ".join(str(x) for x in df.iloc[row,2:27]))
        
    clean_headline=[]
    for i in range(0,len(headline)):
        clean_headline.append(re.sub("b[(')]",'',headline[i])) #remove b'
        clean_headline[i]=re.sub('b[(")]','',clean_headline[i]) #remove b"
        clean_headline[i]=re.sub("\'",'',clean_headline[i]) #remove \'
        
    df['Combined_news'] = clean_headline
    df["tokenized_news"] = df["Combined_news"].apply(lambda x: tokenizer(x, truncation=True, padding='max_length', max_length=512))
    
    return df[['Date', 'Label', 'tokenized_news']]

In [5]:
def preprocess_news(data, row_idx, m, n):
    m_days_news = []
    n_days_news = []

    for i in range(max(m, n)):
        if row_idx - i >= 0:
            daily_news = data.iloc[row_idx - i]
            tokenized_news = daily_news["tokenized_news"]
            tokenized_news_tensor = torch.tensor(tokenized_news['input_ids']).unsqueeze(0)
            if i < m:
                
                m_days_news.append(tokenized_news_tensor)
            if i < n:
                n_days_news.append(tokenized_news_tensor)

    return m_days_news, n_days_news


In [6]:
df_train = combine_text(df_train, tokenizer)
df_valid = combine_text(df_valid, tokenizer)
df_test = combine_text(df_test, tokenizer)

In [7]:
M = 5  # Number of past days for GRU model
N = 7  # Number of past days for Attention model

df_train["input"] = [preprocess_news(df_train, idx, M, N) for idx in range(len(df_train))]
df_valid["input"] = [preprocess_news(df_valid, idx, M, N) for idx in range(len(df_valid))]
df_test["input"] = [preprocess_news(df_test, idx, M, N) for idx in range(len(df_test))]

In [8]:
df_train = df_train[10:]
df_valid = df_valid[10:]
df_test = df_test[10:]

In [9]:
from transformers import AutoModel

# Hybrid Attention Sequential Stock Model
class NewsGRU(nn.Module):
    def __init__(self, pretrained_model_name, hidden_size, num_layers):
        super(NewsGRU, self).__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name)
        self.lstm = nn.LSTM(self.bert.config.hidden_size * 5, hidden_size, num_layers, batch_first=True)
        self.pool = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):
        # x is a tensor of shape [batch_size, num_days, max_length]
        batch_size, num_days, max_length = x.size()

        # Reshape the input to feed it to BERT
        x = x.view(-1, max_length)

        # Get BERT embeddings
        embeddings = self.bert(x).last_hidden_state  # shape: [batch_size * num_days, max_length, hidden_size]

        # Reshape the embeddings to feed them to LSTM
        embeddings = embeddings.view(batch_size, num_days, max_length, self.bert.config.hidden_size).permute(0, 2, 1, 3).contiguous()
        embeddings = embeddings.view(batch_size, max_length, -1)  # shape: [batch_size, max_length, num_days * hidden_size]
        

        # Get the LSTM output
        lstm_output, _ = self.lstm(embeddings)  # shape: [batch_size, max_length, num_days, hidden_size]

        # Pool the LSTM output
        lstm_pooled = self.pool(lstm_output.permute(0, 2, 1)).squeeze(2)  # shape: [batch_size, hidden_size]

        return lstm_pooled

class NewsAttention(nn.Module):
    def __init__(self, pretrained_model_name):
        super(NewsAttention, self).__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name)

    def forward(self, x):
        # x is a tensor of shape [batch_size, num_days, max_length]
        batch_size, num_days, max_length = x.size()

        # Reshape the input to feed it to BERT
        x = x.view(-1, max_length)

        # Get BERT embeddings and attention weights
        outputs = self.bert(x, output_attentions=True)
        
        embeddings = outputs.last_hidden_state  # shape: [batch_size * num_days, max_length, hidden_size]
        attentions = torch.cat(outputs.attentions, dim=1)  # shape: [batch_size * num_days, num_heads * num_layers, max_length, max_length]

        # Calculate the attention-weighted embeddings
        attention_weights = attentions.mean(dim=1)  # shape: [batch_size * num_days, max_length, max_length]
        attention_embedding = torch.bmm(attention_weights, embeddings)  # shape: [batch_size * num_days, max_length, hidden_size]

        # Reshape the embeddings to the original shape
        attention_embedding = attention_embedding.view(batch_size, num_days, max_length, -1)  # shape: [batch_size, num_days, max_length, hidden_size]

        # Pool the attention embeddings
        pooled_attention = attention_embedding.mean(dim=[1, 2])  # shape: [batch_size, hidden_size]

        return pooled_attention
    
    
class Attention(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Attention, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim, bias=False)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        attention_scores = self.fc(x)
        attention_weights = self.softmax(attention_scores)
        context_vector = torch.sum(attention_weights * x, dim=1)
        return context_vector
    
class HASSModel(nn.Module):
    def __init__(self, input_dim, output_dim, dropout):
        super(HASSModel, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, gru_embedding, attention_embedding):
        fused_embedding = torch.cat((gru_embedding, attention_embedding), dim=-1)
        fused_embedding = self.dropout(fused_embedding)
        logits = self.fc(fused_embedding)
        return logits


In [10]:
M = 5
N = 7  # Number of past days to consider
HIDDEN_DIM = 256
OUTPUT_DIM = 2
DROPOUT = 0.3
pretrained_model_name = "distilbert-base-uncased"

# Initialize models
gru_model = NewsGRU(pretrained_model_name, hidden_size=256, num_layers=1)
attention_model = NewsAttention(pretrained_model_name)
HASS = HASSModel(1024, OUTPUT_DIM, DROPOUT)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- T

In [11]:
class NewsDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):

        return len(self.data)

    def __getitem__(self, idx):
        input_data_m, input_data_n = self.data.iloc[idx]["input"]
        label = self.data.iloc[idx]["Label"]

        input_data_m = torch.cat(input_data_m, dim=0)
        input_data_n = torch.cat(input_data_n, dim=0)

        return input_data_m, input_data_n, torch.tensor(label)


In [12]:
BATCH_SIZE = 4

train_dataset = NewsDataset(df_train)
test_dataset = NewsDataset(df_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [13]:

# Set up the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(list(gru_model.parameters()) + list(attention_model.parameters()) + list(HASS.parameters()), lr=1e-5)

# Set the number of epochs
num_epochs = 10

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
gru_model.to(device)
attention_model.to(device)
HASS.to(device)

for epoch in range(num_epochs):
    # Training
    gru_model.train()
    attention_model.train()
    HASS.train()

    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for batch_data_m, batch_data_n, batch_labels in tqdm(train_loader):
        optimizer.zero_grad()

        batch_data_m = batch_data_m.to(device)
        batch_data_n = batch_data_n.to(device)
        batch_labels = batch_labels.to(device)

        gru_embedding = gru_model(batch_data_m)
        attention_embedding = attention_model(batch_data_n)
        logits = HASS(gru_embedding, attention_embedding)

        loss = criterion(logits, batch_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(logits, 1)
        correct_predictions += (predicted == batch_labels).sum().item()
        total_predictions += batch_labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct_predictions / total_predictions

    # Evaluation
    gru_model.eval()
    attention_model.eval()
    HASS.eval()

    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for batch_data_m, batch_data_n, batch_labels in tqdm(test_loader):
            batch_data_m = batch_data_m.to(device)
            batch_data_n = batch_data_n.to(device)
            batch_labels = batch_labels.to(device)

            gru_embedding = gru_model(batch_data_m)
            attention_embedding = attention_model(batch_data_n)
            logits = HASS(gru_embedding, attention_embedding)

            loss = criterion(logits, batch_labels)

            running_loss += loss.item()

            _, predicted = torch.max(logits, 1)
            correct_predictions += (predicted == batch_labels).sum().item()
            total_predictions += batch_labels.size(0)

    test_loss = running_loss / len(test_loader)
    test_accuracy = correct_predictions / total_predictions

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:07<00:00,  6.28it/s]


Epoch 1/10, Train Loss: 0.7022, Train Accuracy: 0.5085, Test Loss: 0.6889, Test Accuracy: 0.5450


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:07<00:00,  6.28it/s]


Epoch 2/10, Train Loss: 0.6954, Train Accuracy: 0.5130, Test Loss: 0.6984, Test Accuracy: 0.4550


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:07<00:00,  6.28it/s]


...

Epoch 8/10, Train Loss: 0.0793, Train Accuracy: 0.9772, Test Loss: 1.1817, Test Accuracy: 0.4868


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:07<00:00,  6.29it/s]


Epoch 9/10, Train Loss: 0.0492, Train Accuracy: 0.9886, Test Loss: 1.2347, Test Accuracy: 0.4868


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:07<00:00,  6.28it/s]

Epoch 10/10, Train Loss: 0.0367, Train Accuracy: 0.9899, Test Loss: 1.3394, Test Accuracy: 0.5238
